In [1]:
import tensorflow as tf
from tensorflow import keras
model = keras.models.load_model('xception_v4_large_08_0.894.h5')
tf.saved_model.save(model, 'clothing-model')

INFO:tensorflow:Assets written to: clothing-model\assets


To be able to use this model later, we need to know a few things:

- The name of the model signature. The model signature describes the model’s inputs and outputs. You can read more about model signatures here: https://www.tensorflow.org/tfx/serving/signature_defs.
- The name of the input layer.
- The name of the output layer.

In [2]:
!saved_model_cli show --dir clothing-model --all

2022-05-28 20:40:07.495392: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-28 20:40:07.495455: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.



MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_8'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 299, 299, 3)
        name: serving_default_input_8:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_7'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          input_8: TensorS

In this output, we’re interested in three things:

- The signature definition (signature_def) of the model. In this case, it’s serving_ default.
- The input (input_8). The name of the input of the model.
- The output (dense_7). The name of the output layer of the model.

```python

docker run -it --rm \
    -p 8500:8500 \
    -v "C:\Users\navan\PGP\deploy-tf\clothing-model:/models/clothing-model/1" \
    -e MODEL_NAME=clothing-model \
    tensorflow/serving:2.3.0

```

When running it, we use three parameters:

* -p: To map port 8500 on the host machine (the computer where we run Docker) to port 8500 inside the container 
* -v: To put the model files inside the Docker image . The model is put in /models/clothing-model/1, where clothing-model is the name of the model and 1 is the version.
* -e: To set the MODEL_NAME variable to clothing-model, which is the directory name from.

## Invoking TfServing from Model

TF Serving uses gRPC—a special protocol designed for high-performance communication. This protocol relies on protobuf, a format for effective data transfer. Unlike JSON, it’s binary, which makes the requests significantly more compact.


### Installation steps are:

```
pip install grpcio==1.32.0
pip install tensorflow-serving-api==2.3.0
pip install keras_image_helper==0.0.1
```

In [5]:
import grpc  # for communicating with tf serving
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [6]:
host = "localhost:8500"
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

#### Reference:

https://pypi.org/project/keras-image-helper/

In [7]:
from keras_image_helper import create_preprocessor
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [8]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [9]:
type(X)

numpy.ndarray

In [10]:
X.shape

(1, 299, 299, 3)

For gRPC, we need to convert it to protobuf. TensorFlow has a special function for that: `tf.make_tensor_proto`

In [11]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data,shape = data.shape)

In [12]:
request = predict_pb2.PredictRequest()
request.model_spec.name = "clothing-model"
request.model_spec.signature_name = "serving_default"
request.inputs["input_8"].CopyFrom(np_to_protobuf(X))

In [13]:
pb_result = stub.Predict(request, timeout = 20.0)

In [19]:
pred = pb_result.outputs['dense_7'].float_val

In [20]:
pred

[-1.8682907819747925, -4.761244773864746, -2.316983461380005, -1.062570333480835, 9.887161254882812, -2.812433958053589, -3.666283130645752, 3.200361728668213, -2.6023383140563965, -4.835046768188477]

In [21]:
labels = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]
 
result = {c: p for c, p in zip(labels, pred)}

In [22]:
result

{'dress': -1.8682907819747925,
 'hat': -4.761244773864746,
 'longsleeve': -2.316983461380005,
 'outwear': -1.062570333480835,
 'pants': 9.887161254882812,
 'shirt': -2.812433958053589,
 'shoes': -3.666283130645752,
 'shorts': 3.200361728668213,
 'skirt': -2.6023383140563965,
 't-shirt': -4.835046768188477}